# Wireguard

> Setting up a **WireGuard VPN on an AWS EC2 instance** to access your home computer securely requires the following steps:


- skip_showdoc:true
- skip_exec: true

### **🛠️ Step 1: Deploy an EC2 Instance**
1. **Go to AWS EC2 Dashboard** → Launch a new instance.
2. **Choose Amazon Linux 2 / Ubuntu 22.04 / Debian 12** (or any Linux distro).
3. **Select Instance Type** (e.g., `t3.micro` for free tier).
4. **Configure Network Settings**:
   - **Assign an Elastic IP** (so it has a fixed public IP).
   - **Security Groups**:
     - Open **UDP port 51820** (default WireGuard port).
     - Open **SSH (port 22)** to your IP for remote access.

5. **Launch & Connect** to the instance via SSH:
   ```sh
   ssh -i your-key.pem ec2-user@your-ec2-public-ip
   ```

### **🛠️ Step 2: Install WireGuard on EC2**
Run the following commands to install WireGuard:

#### **For Amazon Linux 2 / CentOS / RHEL**
```sh
sudo yum install -y wireguard-tools
```

#### **For Ubuntu / Debian**
```sh
sudo apt update && sudo apt install -y wireguard
```

---

### **🛠️ Step 3: Generate WireGuard Keys**
On your **EC2 server**, generate the keys:
```sh
cd /etc/wireguard
umask 077  # Ensures private keys are not world-readable
wg genkey | tee privatekey | wg pubkey > publickey
```

- **`privatekey`** → EC2 private key (keep it safe).
- **`publickey`** → EC2 public key (share it with clients).

---

### **🛠️ Step 4: Configure WireGuard on EC2**
Create the **WireGuard config file**:
```sh
sudo nano /etc/wireguard/wg0.conf
```

Add the following:
```ini
[Interface]
PrivateKey = YOUR_EC2_PRIVATE_KEY
Address = 10.0.0.1/24  # Internal VPN IP for EC2
ListenPort = 51820

# Allow traffic forwarding
PostUp = iptables -A FORWARD -i wg0 -j ACCEPT; iptables -t nat -A POSTROUTING -o eth0 -j MASQUERADE
PostDown = iptables -D FORWARD -i wg0 -j ACCEPT; iptables -t nat -D POSTROUTING -o eth0 -j MASQUERADE

[Peer]
PublicKey = YOUR_HOME_PC_PUBLIC_KEY
AllowedIPs = 10.0.0.2/32  # Home PC internal VPN IP
```

Replace:
- **`YOUR_EC2_PRIVATE_KEY`** with the private key from `privatekey`.
- **`YOUR_HOME_PC_PUBLIC_KEY`** with your home computer’s public key.

---

### **🛠️ Step 5: Enable IP Forwarding**
Edit the sysctl config:
```sh
sudo nano /etc/sysctl.conf
```
Uncomment or add this line:
```ini
net.ipv4.ip_forward = 1
```
Apply the changes:
```sh
sudo sysctl -p
```

---

### **🛠️ Step 6: Start WireGuard on EC2**
Enable and start WireGuard:
```sh
sudo systemctl enable wg-quick@wg0
sudo systemctl start wg-quick@wg0
```
Check status:
```sh
sudo wg
```

---

### **🛠️ Step 7: Install & Configure WireGuard on Your Home PC**
#### **Install WireGuard on Home PC (Windows / Linux)**
- **Windows**: Install [WireGuard for Windows](https://www.wireguard.com/install/)
- **Linux**: Run:
  ```sh
  sudo apt install -y wireguard
  ```

#### **Generate Keys on Home PC**
Run:
```sh
wg genkey | tee privatekey | wg pubkey > publickey
```
- **`privatekey`** → Home PC’s private key.
- **`publickey`** → Home PC’s public key (use this in EC2 config).

#### **Create Home PC Configuration**
Edit **`wg0.conf`**:
```ini
[Interface]
PrivateKey = YOUR_HOME_PC_PRIVATE_KEY
Address = 10.0.0.2/24  # Home PC VPN IP

[Peer]
PublicKey = YOUR_EC2_PUBLIC_KEY
Endpoint = YOUR_EC2_PUBLIC_IP:51820
AllowedIPs = 10.0.0.0/24, 192.168.1.0/24  # Route local network traffic
PersistentKeepalive = 25
```

Replace:
- **`YOUR_HOME_PC_PRIVATE_KEY`** with the private key from `privatekey`.
- **`YOUR_EC2_PUBLIC_KEY`** with the public key from EC2.
- **`YOUR_EC2_PUBLIC_IP`** with your EC2 instance's public IP.

---

### **🛠️ Step 8: Start WireGuard on Home PC**
Run:
```sh
sudo wg-quick up wg0
```
Check connection:
```sh
ping 10.0.0.1
```
If successful, your home PC can now access the EC2 instance.

---

### **🛠️ Step 9: Route Home Network Traffic (Optional)**
To **access devices on your home network**, modify `AllowedIPs` in the home PC config:
```ini
AllowedIPs = 10.0.0.0/24, 192.168.1.0/24
```
Then, on your **home router**, **add a static route**:
- **Destination**: `10.0.0.0/24`
- **Gateway**: Home PC’s local IP (e.g., `192.168.1.100`)

Now you can access home devices (e.g., `ssh 192.168.1.50` for a home server).

---

### **🛠️ Step 10: Automate WireGuard Startup**
On **EC2**:
```sh
sudo systemctl enable wg-quick@wg0
```
On **Home PC**:
```sh
sudo systemctl enable wg-quick@wg0
```
